In [ ]:
# 已经新增8个特征Year、Month、day、week、weekday、season、month_phase、is_holiday
 #"D:/ProgrammingLeaning/JupyterLearning/产品订单processed_data/processed_data0620.csv"   

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np     
import pandas as pd    
from pandas.plotting import autocorrelation_plot as auto_corr
import joblib

import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

import math
from datetime import datetime
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression 
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [55]:
import pandas as pd 
import numpy as np 
#2015zhi201901
data = pd.read_csv("D:/ProgrammingLeaning/JupyterLearning/产品订单processed_data/processed_2015to2019_01_data0622.csv",encoding='utf-8')
data['order_date'] = pd.to_datetime(data['order_date'])
data = data.sort_values(by='order_date')
data

,Unnamed: 0,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,Year,Month,day,week,weekday,season,month_phase,is_holiday
0,0,2015-09-01,104,22069,307,403,offline,1114.0,19,2015,9,1,36,1,Fall,Early Month,False
1,1,2015-09-01,104,20028,301,405,offline,1012.0,12,2015,9,1,36,1,Fall,Early Month,False
11,11,2015-09-02,104,20606,308,404,offline,1615.0,17,2015,9,2,36,2,Fall,Early Month,False
10,10,2015-09-02,104,21183,307,403,online,428.0,109,2015,9,2,36,2,Fall,Early Month,False
9,9,2015-09-02,104,20448,308,404,online,962.0,3,2015,9,2,36,2,Fall,Early Month,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705614,705614,2019-01-31,102,20693,308,404,NaN,NaN,0,2019,1,31,5,3,Winter,End of Month,False
705615,705615,2019-01-31,102,20696,305,412,NaN,NaN,0,2019,1,31,5,3,Winter,End of Month,False
705616,705616,2019-01-31,102,20697,304,409,NaN,NaN,0,2019,1,31,5,3,Winter,End of Month,False
705598,705598,2019-01-31,102,20649,308,404,NaN,NaN,0,2019,1,31,5,3,Winter,End of Month,False


In [71]:
# 标记促销日
def mark_promotions(data):
    """
    标记促销日。
    :param data: pandas DataFrame
    :return: 标记了促销日的数据框
    """
    data["order_date"] = pd.to_datetime(data["order_date"])
    promo_dates = {(1, 1), (2, 14), (3, 8), (4, 1), (5, 1), (6, 1), (6, 18), (11, 11), (12, 12)}  # 情人节、妇女节、618、双十一、双十二
    # 标记促销日
    data["is_promo"] = data["order_date"].apply(
        lambda x: (x.month, x.day) in promo_dates
    )
    return data

# 添加滞后特征
def add_lag_features(df, lags=[1, 2, 3, 6, 12, 24, 36, 48, 60]):
    """
    添加滞后特征。
    :param df: pandas DataFrame
    :param lags: list, 滞后步长
    :return: pandas DataFrame with lag features added
    """
    for lag in lags:
        df['ord_qty_lag_' + str(lag)] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'], as_index=False)['ord_qty'].shift(lag).astype(np.float16)
    return df

# 添加均值编码特征
def add_mean_encoding_features(df):
    """
    添加均值编码功能。
    :param df: pandas DataFrame
    :return: pandas DataFrame with mean encoding features added
    """
    df['item_code_avg'] = df.groupby('item_code')['ord_qty'].transform('mean').astype(np.float16)
    df['sales_region_code_need_avg'] = df.groupby('sales_region_code')['ord_qty'].transform('mean').astype(np.float16)
    df['first_cate_code_need_avg'] = df.groupby('first_cate_code')['ord_qty'].transform('mean').astype(np.float16)
    df['second_cate_code_need_avg'] = df.groupby('second_cate_code')['ord_qty'].transform('mean').astype(np.float16)
    df['sales_region_code_item_code_need_avg'] = df.groupby(['item_code', 'sales_region_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['first_cate_code_item_code_need_avg'] = df.groupby(['first_cate_code', 'item_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['second_cate_code_item_code_need_avg'] = df.groupby(['second_cate_code', 'item_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['sales_region_code_first_cate_code_need_avg'] = df.groupby(['sales_region_code', 'first_cate_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['first_cate_code_second_cate_code_need_avg'] = df.groupby(['first_cate_code', 'second_cate_code'])['ord_qty'].transform('mean').astype(np.float16)

    df['year_month_need_avg'] = df.groupby(['Year', 'Month'])['ord_qty'].transform('mean').astype(np.float16)
    df['year_sales_region_code_need_avg'] = df.groupby(['Year', 'sales_region_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['year_first_cate_code_need_avg'] = df.groupby(['Year', 'first_cate_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['year_second_cate_code_need_avg'] = df.groupby(['Year', 'second_cate_code'])['ord_qty'].transform('mean').astype(np.float16)

    df['month_sales_region_code_need_avg'] = df.groupby(['Month', 'sales_region_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['month_first_cate_code_need_avg'] = df.groupby(['Month', 'first_cate_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['month_second_cate_code_need_avg'] = df.groupby(['Month', 'second_cate_code'])['ord_qty'].transform('mean').astype(np.float16)

    df['month_phase_sales_region_code_need_avg'] = df.groupby(['sales_region_code', 'month_phase'])['ord_qty'].transform('mean').astype(np.float16)
    
    return df

# 添加滚动和扩展功能
def add_rolling_and_expanding_features(df):
    """
    添加滚动和扩展功能。
    :param df: pandas DataFrame
    :return: pandas DataFrame with rolling and expanding features added
    """
    df['rolling_need_mean'] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])['ord_qty'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    df['expanding_need_mean'] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])['ord_qty'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    return df

#需求趋势
def add_demand_trend_feature(df):
    """
    Adds a demand trend feature to the DataFrame.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing sales data with the necessary columns.
    
    Returns:
    pd.DataFrame: DataFrame with the demand trend feature added.
    """
    # 日需求
    #df['daily_avg_demand'] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code', 'order_date'])['ord_qty'].transform('mean').astype(np.float16)
    df['daily_avg_demand'] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code', 'order_date'])['ord_qty'].transform(lambda x: x.shift(1).expanding().mean()).astype(np.float16)
    
    # 平均需求
    #df['avg_demand'] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])['ord_qty'].transform('mean').astype(np.float16)
    df['avg_demand'] = df.groupby(['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])['ord_qty'].transform(lambda x: x.shift(1).expanding().mean()).astype(np.float16)
    
    # 需求趋势
    #df['demand_trend'] = (df['daily_avg_demand'] - df['avg_demand']).astype(np.float16)
    df['demand_trend'] = (df['daily_avg_demand'] - df['avg_demand']).astype(np.float16)
    
    df.drop(['daily_avg_demand', 'avg_demand'], axis=1, inplace=True)
    
    return df

# Example usage:
# df = add_demand_trend_feature(df)

In [72]:
def add_day_index_column(df, date_column, start_date="2015-09-01", end_date="2019-01-31"):
    """
    为数据框添加一个表示从起始日期开始的第几天的“D”列。
    
    参数:
    df (pandas.DataFrame): 数据框
    date_column (str): 日期列的名称
    start_date (str): 起始日期，格式为"YYYY-MM-DD"
    end_date (str): 结束日期，格式为"YYYY-MM-DD"
    
    返回:
    pandas.DataFrame: 添加了“D”列的数据框
    """
    # 创建日期范围
    dt1 = pd.date_range(start=start_date, end=end_date, freq="D")
    
    # 创建日期到第几天的映射字典
    dicts = {}
    for i, date in enumerate(dt1, start=1):
        dicts[str(date).split(' ')[0]] = i
    
    # 将日期列转换为字符串格式
    df[date_column] = df[date_column].astype(str)
    
    # 添加“D”列
    df['D'] = df[date_column].map(dicts)
    
    return df


In [73]:

# 特征工程主函数，调用所有特征工程函数
def feature_engineering_pipeline(data, start_date, end_date):
    """
    特征工程主函数，应用所有特征工程函数。
    :param data: pandas DataFrame, 输入数据
    :param start_date: str, 时间索引特征的开始日期
    :param end_date: str, 时间索引特征的结束日期
    :return: pandas DataFrame, 添加所有特征后的数据
    """
    # 添加时间索引特征
    data = add_day_index_column(data,'order_date')
    
    # 添加滞后特征
    data = add_lag_features(data)
    
    # 添加均值编码特征
    data = add_mean_encoding_features(data)
    
    # 添加滑动窗口统计和开窗数据拓展特征
    data = add_rolling_and_expanding_features(data)
    
    #需求趋势
    data = add_demand_trend_feature(data)
    
    # 标记促销日特征
    data = mark_promotions(data)
    
    return data


def main():
    # 加载数据
    file_path = "D:/ProgrammingLeaning/JupyterLearning/产品订单processed_data/processed_2015to2019_01_data0622.csv"
    data = pd.read_csv(file_path)
    date_column = "order_date"  # 日期列
    
    # 转换日期列
    data[date_column] = pd.to_datetime(data[date_column])
    
    # 去重处理
    data.drop_duplicates(inplace=True)
    
    # 定义时间索引特征的开始和结束日期
    start_date = '2015-09-01'
    end_date = '2019-01-31'
    
    # 应用特征工程主函数
    processed_data = feature_engineering_pipeline(data, start_date, end_date)
    print("Feature engineering is done!")
    
    # 保存处理后的数据
    processed_data.to_csv("D:/ProgrammingLeaning/JupyterLearning/产品订单processed_data/test01_feature_engineering2015-201901_data0622.csv", index=False)
    #print("Processed data is saved in folder 产品订单processed_2019_data")
    return processed_data

if __name__ == "__main__":
    main()


Feature engineering is done!


In [68]:
data = pd.read_csv("D:/ProgrammingLeaning/JupyterLearning/产品订单processed_data/feature_engineering2015-201901_data0622.csv")
data

,Unnamed: 0,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,Year,...,year_first_cate_code_need_avg,year_second_cate_code_need_avg,month_sales_region_code_need_avg,month_first_cate_code_need_avg,month_second_cate_code_need_avg,month_phase_sales_region_code_need_avg,rolling_need_mean,expanding_need_mean,demand_trend,is_promo
0,0,2015-09-01,104,22069,307,403,offline,1114.0,19,2015,...,116.60,116.600,103.75,110.75,110.75,92.60,NaN,NaN,6.710,False
1,1,2015-09-01,104,20028,301,405,offline,1012.0,12,2015,...,84.44,84.440,103.75,70.44,70.44,92.60,NaN,NaN,3.000,False
2,2,2015-09-02,102,20457,305,412,offline,164.0,308,2015,...,92.80,92.800,73.06,82.44,82.44,79.25,NaN,NaN,141.400,False
3,3,2015-09-02,101,20657,303,410,offline,2996.0,18,2015,...,52.70,9.664,87.10,52.47,17.38,85.75,NaN,NaN,6.890,False
4,4,2015-09-02,102,21350,305,412,offline,267.0,107,2015,...,92.80,92.800,73.06,82.44,82.44,79.25,NaN,NaN,-35.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707354,707354,2019-01-31,105,22066,307,403,NaN,NaN,0,2019,...,0.00,0.000,35.94,44.75,44.75,67.94,0.0,257.000,-257.000,False
707355,707355,2019-01-31,105,22072,305,412,NaN,NaN,0,2019,...,0.00,0.000,35.94,33.70,33.70,67.94,0.0,37.250,-37.250,False
707356,707356,2019-01-31,105,22075,307,403,NaN,NaN,0,2019,...,0.00,0.000,35.94,44.75,44.75,67.94,0.0,0.000,0.000,False
707357,707357,2019-01-31,105,22083,303,401,NaN,NaN,0,2019,...,0.00,0.000,35.94,18.80,25.14,67.94,0.0,60.970,-60.970,False
